# Finantial Times Scraping

### Inicializações

In [11]:
import pandas as pd
from datetime import datetime, timedelta

### Scrapping Exchange Rates from www.exchangerates.org.uk

In [12]:
url = 'http://www.exchangerates.org.uk/USD-EUR-exchange-rate-history.html'
csv = 'exchange_rates.csv'
#csv = 'D:\Personal\Python\Fundos\exchange_rates.csv'

df = pd.read_html(url, skiprows=[0,1,182])[0][[0,1]].rename(columns={0:'Date',1:'USD2EUR'})
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x, infer_datetime_format = True).date())
df['USD2EUR'] = df['USD2EUR'].apply(lambda x: float(x[8:15]))
df.set_index('Date',inplace=True)

df_historic_usd2eur = pd.read_csv(csv,sep=';',parse_dates=True)
df_historic_usd2eur['Date'] = df_historic_usd2eur['Date'].apply(lambda x: pd.to_datetime(x, infer_datetime_format = True).date())
df_historic_usd2eur.set_index('Date',inplace=True)

df_historic_usd2eur = pd.concat([df.loc[:max(df_historic_usd2eur.index)][:-1],df_historic_usd2eur])
df_historic_usd2eur.to_csv(csv,sep=';')

### Update Portofolio From Transactions

In [13]:
#transacoes_csv = 'D:\\Personal\\Python\\Fundos\\transacoes.csv'
transacoes_csv = 'transacoes.csv'

encoding='latin_1'
thousands = ','
decimal = '.'
to_date = lambda d: datetime.strptime(d, '%d-%m-%Y')
converters={'Data de subscrição': to_date}

portfolio_csv = 'portofolio.csv'
df_portofolio = pd.read_csv(transacoes_csv ,sep=',',encoding=encoding,thousands=thousands, decimal=decimal, converters=converters)
df_portofolio = df_portofolio[pd.isnull(df_portofolio['Data de resgate'])].groupby(['Code','Nome','Moeda'])['Quantidade'].sum().reset_index()
df_portofolio.to_csv(portfolio_csv,sep=';',index=False)
df_portofolio.sort_values(by='Nome')

,Code,Nome,Moeda,Quantidade
21,LU0524465977,Alken European Opport A,EUR,26.86495
36,LU1997245177,Allianz China A Shares AT USD,USD,257.82900
34,LU1883314244,Amundi Europ Value A C,EUR,24.05600
22,LU0628613639,BGF World Resources E2,EUR,1251.44000
5,LU0108459040,Candriam Biotech C Cap USD,USD,3.60500
29,LU1278917452,DWS CROCI Sect Plus LC,EUR,40.95800
33,LU1769942746,DWS CROCI US Divid,USD,8.51300
23,LU0740823439,DWS Invest German Equities,EUR,5.19500
17,LU0300742037,F Natural Rosources N,EUR,729.92700
8,LU0140363697,F Technology N,EUR,179.27600


### Update wiht scrapping Quotes from FT 

In [14]:
url = 'https://markets.ft.com/data/funds/tearsheet/historical?s={}:{}'
xls = 'historico_cotacoes.xlsx'
to_date = lambda d: datetime.strptime(d, '%Y-%m-%d')
converters={'date': to_date}

novo_dict_df = dict()

dict_df = {key.strip():value for key,value in pd.read_excel(xls,sheet_name=None).items()}

for symbol, df in dict_df.items():
    if symbol in df_portofolio['Code'].tolist():
        dict_df[symbol]['Date'] = dict_df[symbol]['Date'].apply(lambda x: x.date())
        novo_dict_df[symbol] = dict_df[symbol].set_index('Date',drop=True).sort_index(ascending=False)
    

for symbol, _, currency, _ in df_portofolio.itertuples(index=False):

    # try to get data from a csv dwoloaded from Morningstar
    try:
        path = '.\\cotacoes_morningstar\\{}.csv'.format(symbol)
        df = pd.read_csv(path ,sep=';',encoding=encoding,thousands=thousands, decimal=decimal, converters=converters)[['date','price']]
        df['date'] = df['date'].apply(lambda x: x.date())
        df = df.set_index('date',drop=True).sort_index(ascending=False).rename(columns={'price':'Close'})
        df.index.rename('Date',inplace=True)
        if symbol in dict_df.keys():
            novo_dict_df[symbol] = pd.concat([novo_dict_df[symbol], df.loc[df.index.difference(novo_dict_df[symbol].index)]]).sort_index(ascending=False)
        else:
            novo_dict_df[symbol] = df
    except:
        print("Warning: No "+path)

    # try to data from FT
    try:
        url = 'https://markets.ft.com/data/funds/tearsheet/historical?s={}:{}'.format(symbol,currency)
        df = pd.read_html(url)[0]
        df['Date'] = df['Date'].apply(lambda x: x[:-17]).apply(lambda x: pd.to_datetime(x, infer_datetime_format = True).date())
        df = df.set_index('Date',drop=True)[['Close']]
        novo_dict_df[symbol] = pd.concat([novo_dict_df[symbol], df.loc[df.index.difference(novo_dict_df[symbol].index)]]).sort_index(ascending=False)
    except:
        print("Warning: Invalid "+url)
        

In [15]:
with pd.ExcelWriter(xls, engine="openpyxl") as writer:
    for symbol, df in novo_dict_df.items():
        df.to_excel(writer, sheet_name=symbol)
print(datetime.now())

2023-09-09 20:09:21.245563


In [16]:
df_portofolio.sort_values(by='Nome')

,Code,Nome,Moeda,Quantidade
21,LU0524465977,Alken European Opport A,EUR,26.86495
36,LU1997245177,Allianz China A Shares AT USD,USD,257.82900
34,LU1883314244,Amundi Europ Value A C,EUR,24.05600
22,LU0628613639,BGF World Resources E2,EUR,1251.44000
5,LU0108459040,Candriam Biotech C Cap USD,USD,3.60500
29,LU1278917452,DWS CROCI Sect Plus LC,EUR,40.95800
33,LU1769942746,DWS CROCI US Divid,USD,8.51300
23,LU0740823439,DWS Invest German Equities,EUR,5.19500
17,LU0300742037,F Natural Rosources N,EUR,729.92700
8,LU0140363697,F Technology N,EUR,179.27600
